In [192]:
import game

In [194]:
game._build_row_tables()

start_board = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 16, 0],
    [0, 0, 16, 0],
]
bs = game.to_bitset(start_board)

game.print_board(start_board, label="Start:")
print("Bitset:", bs, "\n")

for name, fn in ("left", game.left), ("right", game.right), ("up", game.up), ("down", game.down):
    bs = fn(bs)
    print(f"After {name}:")
    game.print_board(game.to_board(bs))
    print("Bitset:", bs, "\n")


Start:
     .      .      .      .
     .      .      .      .
     .      .  65536      .
     .      .  65536      .
Bitset: 18889483945877090336768 

After left:
     .      .      .      .
     .      .      .      .
 65536      .      .      .
 65536      .      .      .
Bitset: 18446761665895596032 

After right:
     .      .      .      .
     .      .      .      .
     .      .      .  65536
     .      .      .  65536
Bitset: 604463486268066890776576 

After up:
      .       .       .  131072
      .       .       .       .
      .       .       .       .
      .       .       .       .
Bitset: 557056 

After down:
      .       .       .       .
      .       .       .       .
      .       .       .       .
      .       .       .  131072
Bitset: 642241841670271749062656 

